### Домашнее задание №9

Реализовать API для модели машинного обучения (можно взять модели из второй или третьей лекции), код представить в виде пулл реквеста в репозиторий курса (https://github.com/MindSetLib/geekbrains_tasks).

Пример API из лекции 4 - https://github.com/MindSetLib/Flask-ML-API

1. Cоздим модель на базе датасета freMPL-R.csv, проверим, протестим и сохраним; 
2. Создим скрипт, который принимает через response get входящие параметры для модели, потом запускает уже обученную модель и дает прогноз через post; 
3. Под этот скрипт обеспечиваем архитектуру через flask и html прописанную страницу.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Загрузим набор данных

df = pd.read_csv('freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           115155 non-null  float64
 1   LicAge             115155 non-null  int64  
 2   RecordBeg          115155 non-null  object 
 3   RecordEnd          59455 non-null   object 
 4   Gender             115155 non-null  object 
 5   MariStat           115155 non-null  object 
 6   SocioCateg         115155 non-null  object 
 7   VehUsage           115155 non-null  object 
 8   DrivAge            115155 non-null  int64  
 9   HasKmLimit         115155 non-null  int64  
 10  BonusMalus         115155 non-null  int64  
 11  ClaimAmount        115155 non-null  float64
 12  ClaimInd           115155 non-null  int64  
 13  ClaimNbResp        115155 non-null  float64
 14  ClaimNbNonResp     115155 non-null  float64
 15  ClaimNbParking     115155 non-null  float64
 16  Cl

In [3]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.head()

Unique values of ClaimInd: [0]


,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [4]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

In [5]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [6]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [7]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


In [8]:
list(df.VehUsage.unique())

['Professional', 'Private+trip to office', 'Private', 'Professional run']

In [9]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


In [10]:
df['SocioCateg'].unique()

array(['CSP50', 'CSP55', 'CSP60', 'CSP48', 'CSP6', 'CSP66', 'CSP1',
       'CSP46', 'CSP21', 'CSP47', 'CSP42', 'CSP37', 'CSP22', 'CSP3',
       'CSP49', 'CSP20', 'CSP2', 'CSP40', 'CSP7', 'CSP26', 'CSP65',
       'CSP41', 'CSP17', 'CSP57', 'CSP56', 'CSP38', 'CSP51', 'CSP59',
       'CSP30', 'CSP44', 'CSP61', 'CSP63', 'CSP45', 'CSP16', 'CSP43',
       'CSP39', 'CSP5', 'CSP32', 'CSP35', 'CSP73', 'CSP62', 'CSP52',
       'CSP27', 'CSP24', 'CSP19', 'CSP70'], dtype=object)

In [11]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [12]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

,Frequency
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [13]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

In [14]:
df = df.select_dtypes(exclude=['object'])

In [15]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,...,VehUsage_Professional,VehUsage_Professional run,SocioCateg_CSP1,SocioCateg_CSP2,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq
0,0.083,332,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
1,0.916,333,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
2,0.550,173,0,0,32,0,68,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,1024
3,0.089,364,1,0,52,0,50,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,2704
4,0.233,426,0,0,57,0,50,0.0,0,0.0,...,0,0,0,0,0,0,0,1,0,3249


In [16]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [17]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

,Policies
0.0,104286
2.0,3529
1.0,3339
3.0,2310
4.0,1101
5.0,428
6.0,127
7.0,26
8.0,6
9.0,2


In [18]:
dfAC = df[df.ClaimsCount > 0].copy()
dfAC['AvgClaim'] = dfAC.ClaimAmount/dfAC.ClaimsCount

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
# Разбиение датасета для частоты на train/val/test

x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount'], axis=1), df.ClaimsCount, test_size=0.3, random_state=1)
x_valid_c, x_test_c, y_valid_c, y_test_c = train_test_split(x_test_c, y_test_c, test_size=0.5, random_state=1)

In [21]:
# Разбиение датасета для среднего убытка на train/val/test 

x_train_ac, x_test_ac, y_train_ac, y_test_ac = train_test_split(dfAC.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount', 'AvgClaim'], axis=1), dfAC.AvgClaim, test_size=0.3, random_state=1)
x_valid_ac, x_test_ac, y_valid_ac, y_test_ac = train_test_split(x_test_ac, y_test_ac, test_size=0.5, random_state=1)

In [22]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 11.0.6+8-b765.1, mixed mode)
  Starting server from C:\Users\fini1\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\fini1\AppData\Local\Temp\tmpcnxiiqcv
  JVM stdout: C:\Users\fini1\AppData\Local\Temp\tmpcnxiiqcv\h2o_fini1_started_from_python.out
  JVM stderr: C:\Users\fini1\AppData\Local\Temp\tmpcnxiiqcv\h2o_fini1_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Yekaterinburg
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_fini1_6aeers
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.492 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


### Построение GLM для частоты страховых случаев

In [23]:
# Преобразование в H2O-Frame

h2o_train_c = h2o.H2OFrame(pd.concat([x_train_c, y_train_c], axis=1))
h2o_valid_c = h2o.H2OFrame(pd.concat([x_valid_c, y_valid_c], axis=1))
h2o_test_c = h2o.H2OFrame(pd.concat([x_test_c, y_test_c], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [24]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_poisson = H2OGeneralizedLinearEstimator(family = "poisson", link = "Log", nfolds=5)
glm_poisson.train(y="ClaimsCount", x = h2o_train_c.names[1:-1], training_frame = h2o_train_c, validation_frame = h2o_valid_c, weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [25]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_poisson.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,poisson,log,"Elastic Net (alpha = 0.5, lambda = 1.336E-4 )",20,19,3,Key_Frame__upload_8b8c85749eeef5c44850645ca62b4956.hex


In [26]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_poisson.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.5032792,0.0054369764,0.50411516,0.5091655,0.49432978,0.50399107,0.5047944
1,mean_residual_deviance,1.2440214,0.02325139,1.251381,1.2599111,1.2028998,1.2513491,1.2545658
2,mse,0.7542519,0.022519989,0.7621492,0.7625072,0.71412086,0.7664669,0.7660153
3,null_deviance,9087.195,260.46765,9062.951,9275.309,8660.452,9315.427,9121.839
4,r2,0.011615338,0.0018254336,0.00917822,0.01425713,0.011530507,0.011110533,0.012000298
5,residual_deviance,8871.977,257.21805,8893.538,9032.646,8438.215,9094.131,8901.354
6,rmse,0.8683978,0.013096443,0.8730116,0.87321657,0.8450567,0.87548095,0.875223
7,rmsle,0.41021323,0.0037265676,0.41111726,0.41366634,0.40383446,0.41102904,0.411419


In [27]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_poisson._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.220675,-1.285319
1,LicAge,0.000194,0.030931
2,Gender,0.003870,0.001874
3,MariStat,-0.115084,-0.041152
4,DrivAge,0.004924,0.073696
5,HasKmLimit,-0.467990,-0.145415
6,BonusMalus,0.012914,0.194712
7,OutUseNb,0.084635,0.056781
8,RiskArea,0.015406,0.034085
9,VehUsage_Private,-0.200181,-0.094939


In [28]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_poisson.coef_norm()
for x in range(len(glm_poisson.cross_validation_models())):
    pmodels[x] = glm_poisson.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,-1.28532,-1.28777,-1.29365,-1.27446,-1.28461,-1.28771
LicAge,0.03093,0.01573,0.01842,0.06484,0.04447,0.01635
Gender,0.00187,-0.01346,0.00062,0.00137,0.01260,0.00864
MariStat,-0.04115,-0.04342,-0.04770,-0.04376,-0.03803,-0.03489
DrivAge,0.07370,0.09789,0.01326,0.02104,0.00000,0.15061
HasKmLimit,-0.14541,-0.15368,-0.14584,-0.13434,-0.13360,-0.16044
BonusMalus,0.19471,0.19763,0.18724,0.19642,0.20076,0.18834
OutUseNb,0.05678,0.05522,0.04819,0.06468,0.05832,0.05741
RiskArea,0.03408,0.02427,0.03768,0.03943,0.02926,0.03994
VehUsage_Private,-0.09494,-0.10173,-0.09653,-0.09848,-0.08808,-0.08047


In [29]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

c_train_pred = glm_poisson.predict(h2o_train_c).as_data_frame()
c_valid_pred = glm_poisson.predict(h2o_valid_c).as_data_frame()
c_test_pred = glm_poisson.predict(h2o_test_c).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [30]:
# Сохранение обученной модели

model_glm_poisson = h2o.save_model(model=glm_poisson, path="", force=True)

In [31]:
model_glm_poisson

'C:\\Users\\fini1\\ML_business\\lesson_9\\GLM_model_python_1589124379461_1'

### Построение GLM для среднего убытка

In [32]:
# Преобразование в H2O-Frame

h2o_train_ac = h2o.H2OFrame(pd.concat([x_train_ac, y_train_ac], axis=1))
h2o_valid_ac = h2o.H2OFrame(pd.concat([x_valid_ac, y_valid_ac], axis=1))
h2o_test_ac = h2o.H2OFrame(pd.concat([x_test_ac, y_test_ac], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [33]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_gamma = H2OGeneralizedLinearEstimator(family = "gamma", link = "Log", nfolds=5)
glm_gamma.train(y="AvgClaim", x = h2o_train_ac.names[1:-1], training_frame = h2o_train_ac, validation_frame = h2o_valid_ac, weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [34]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_gamma.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,gamma,log,"Elastic Net (alpha = 0.5, lambda = 1.784E-4 )",20,18,3,Key_Frame__upload_b557e0d60c2d87e781c15eabbf2d48c8.hex


In [35]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_gamma.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,1043.7075,28.375679,1069.2689,1051.4476,1038.8086,1061.7559,997.25665
1,mean_residual_deviance,1.9576064,0.08410336,2.0813854,1.9306284,1.9497277,1.977289,1.8490018
2,mse,7291414.5,6532670.0,4420771.0,1.8889556E7,5035175.5,5034660.5,3076910.0
3,null_deviance,1786.6909,63.274708,1849.4745,1790.9615,1808.9508,1803.559,1680.5085
4,r2,-0.0011706458,0.0028766105,-0.0018459366,8.950012E-4,0.0011415019,-1.779269E-4,-0.005865869
5,residual_deviance,1787.0244,62.499424,1859.8927,1776.8057,1799.9711,1809.166,1689.2869
6,rmse,2538.1223,1030.3821,2102.563,4346.212,2243.9197,2243.805,1754.1123
7,rmsle,1.7135468,0.046912346,1.7900244,1.6632597,1.7162296,1.6982647,1.6999555


In [36]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_gamma._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,7.447459,6.923914
1,LicAge,-0.000361,-0.057896
2,Gender,-0.044813,-0.021696
3,MariStat,0.152243,0.056025
4,DrivAge,-0.026294,-0.389649
5,HasKmLimit,-0.086212,-0.021160
6,BonusMalus,-0.003733,-0.062010
7,OutUseNb,0.005180,0.003776
8,RiskArea,0.040690,0.091382
9,VehUsage_Private,0.044360,0.020112


In [37]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_gamma.coef_norm()
for x in range(len(glm_gamma.cross_validation_models())):
    pmodels[x] = glm_gamma.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,6.92391,6.91802,6.91774,6.92680,6.91270,6.93770
LicAge,-0.05790,0.00547,-0.08551,-0.08732,-0.10264,-0.01749
Gender,-0.02170,-0.01750,-0.01790,-0.00026,-0.03500,-0.03819
MariStat,0.05603,0.07281,0.01277,0.06805,0.06411,0.06124
DrivAge,-0.38965,-0.51748,-0.30051,-0.27292,-0.43904,-0.41885
HasKmLimit,-0.02116,-0.03135,-0.02725,-0.01719,-0.01090,-0.01680
BonusMalus,-0.06201,-0.07375,-0.04229,-0.05554,-0.07306,-0.06198
OutUseNb,0.00378,0.00999,-0.00474,0.00538,0.01802,-0.00778
RiskArea,0.09138,0.07424,0.08891,0.10112,0.07786,0.10987
VehUsage_Private,0.02011,0.03288,0.02425,0.00731,0.02301,-0.00743


In [38]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ac_train_pred = glm_gamma.predict(h2o_train_ac).as_data_frame()
ac_valid_pred = glm_gamma.predict(h2o_valid_ac).as_data_frame()
ac_test_pred = glm_gamma.predict(h2o_test_ac).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [39]:
# Сохранение обученной модели

model_glm_gamma = h2o.save_model(model=glm_gamma, path="", force=True)

In [40]:
model_glm_gamma

'C:\\Users\\fini1\\ML_business\\lesson_9\\GLM_model_python_1589124379461_2'

#### Результаты запуска back-end server

In [ ]:
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\Scripts\python.exe C:/Users/fini1/PycharmProjects/ML_business_hw9/app.py
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) Client VM (build 25.251-b08, mixed mode)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\backend\server.py:385: UserWarning:   You have a 32-bit version of Java. H2O works best with 64-bit Java.
  Please download the latest 64-bit Java SE JDK from Oracle.

  warn("  You have a 32-bit version of Java. H2O works best with 64-bit Java.\n"
  Starting server from C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\fini1\AppData\Local\Temp\tmp73zjsure
  JVM stdout: C:\Users\fini1\AppData\Local\Temp\tmp73zjsure\h2o_fini1_started_from_python.out
  JVM stderr: C:\Users\fini1\AppData\Local\Temp\tmp73zjsure\h2o_fini1_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
--------------------------  ---------------------------------------------------------
H2O_cluster_uptime:         05 secs
H2O_cluster_timezone:       Asia/Yekaterinburg
H2O_data_parsing_timezone:  UTC
H2O_cluster_version:        3.30.0.2
H2O_cluster_version_age:    13 days
H2O_cluster_name:           H2O_from_python_fini1_cqf9hj
H2O_cluster_total_nodes:    1
H2O_cluster_free_memory:    247.5 Mb
H2O_cluster_total_cores:    0
H2O_cluster_allowed_cores:  0
H2O_cluster_status:         accepting new members, healthy
H2O_connection_url:         http://127.0.0.1:54321
H2O_connection_proxy:       {"http": null, "https": null}
H2O_internal_security:      False
H2O_API_Extensions:         Amazon S3, Algos, AutoML, Core V3, TargetEncoder, Core V4
Python_version:             3.6.5 final
--------------------------  ---------------------------------------------------------
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: on
INFO:werkzeug: * Restarting with stat
Checking whether there is an H2O instance running at http://localhost:54321 . connected.
--------------------------  ---------------------------------------------------------
H2O_cluster_uptime:         16 secs
H2O_cluster_timezone:       Asia/Yekaterinburg
H2O_data_parsing_timezone:  UTC
H2O_cluster_version:        3.30.0.2
H2O_cluster_version_age:    13 days
H2O_cluster_name:           H2O_from_python_fini1_cqf9hj
H2O_cluster_total_nodes:    1
H2O_cluster_free_memory:    238.2 Mb
H2O_cluster_total_cores:    4
H2O_cluster_allowed_cores:  4
H2O_cluster_status:         locked, healthy
H2O_connection_url:         http://localhost:54321
H2O_connection_proxy:       {"http": null, "https": null}
H2O_internal_security:      False
H2O_API_Extensions:         Amazon S3, Algos, AutoML, Core V3, TargetEncoder, Core V4
Python_version:             3.6.5 final
--------------------------  ---------------------------------------------------------
WARNING:werkzeug: * Debugger is active!
INFO:werkzeug: * Debugger PIN: 253-168-901
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:__main__:[2020-May-12 14:24:18] request from 127.0.0.1: {'ID': '34', 'LicAge': 11.0, 'RecordBeg': '2004-01-01', 'RecordEnd': '', 'VehAge': 9.0, 'Gender': 'Male', 'MariStat': 'Alone', 'SocioCateg': 'CSP50', 'VehUsage': 'Private', 'DrivAge': 40.0, 'HasKmLimit': 0, 'BonusMalus': 0.05, 'OutUseNb': 0, 'RiskArea': 0}
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'VehUsage_Private': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'VehUsage_Private+trip to office': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'VehUsage_Professional': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'VehUsage_Professional run': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'SocioCateg_CSP1': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'SocioCateg_CSP2': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'SocioCateg_CSP3': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'SocioCateg_CSP4': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'SocioCateg_CSP5': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'SocioCateg_CSP6': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'SocioCateg_CSP7': substituting in a column of NaN
  warnings.warn(w)
C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\lib\site-packages\h2o\job.py:70: UserWarning: Test/Validation dataset is missing column 'DrivAgeSq': substituting in a column of NaN
  warnings.warn(w)
glm prediction progress: |████████████████████████████████████████████████| 100%
INFO:__main__:[2020-May-12 14:24:51] predicted for 33.330455 msec: {'ID': '34', 'value_Poisson': 0.09620912167471554, 'value_Gamma': 1452.692151621146, 'value_BurningCost': 139.76223597122316}

INFO:werkzeug:127.0.0.1 - - [12/May/2020 14:24:51] "POST /predict HTTP/1.1" 200 -


#### Результаты запуска front-end server

C:\Users\fini1\PycharmProjects\ML_business_hw9\venv\Scripts\python.exe C:/Users/fini1/PycharmProjects/ML_business_hw9/front-server/front_server.py
 * Serving Flask app "front_server" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: on
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 253-168-901
 * Running on http://127.0.0.2:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/May/2020 14:23:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2020 14:23:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2020 14:23:48] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/May/2020 14:23:48] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/May/2020 14:23:53] "GET /predict_form HTTP/1.1" 200 -
127.0.0.1 - - [12/May/2020 14:24:51] "POST /predict_form HTTP/1.1" 302 -
{'ID': '34', 'value_BurningCost': 139.76223597122316, 'value_Gamma': 1452.692151621146, 'value_Poisson': 0.09620912167471554}
127.0.0.1 - - [12/May/2020 14:24:51] "GET /predicted/%7B%0A%20%20%22ID%22:%20%2234%22%2C%20%0A%20%20%22value_BurningCost%22:%20139.76223597122316%2C%20%0A%20%20%22value_Gamma%22:%201452.692151621146%2C%20%0A%20%20%22value_Poisson%22:%200.09620912167471554%0A%7D%0A HTTP/1.1" 200 -
